In [ ]:
#Importing useful libraries
import numpy as np
from PIL import Image

import os
import pandas as pd
from tqdm import tqdm
import pickle
import random
import pandas as pd
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
from torchvision.models import Inception3
import pandas as pd
import os

import numpy as np
import pandas as pd
import sklearn
from functools import reduce
import random
from sklearn import svm
import pandas as pd
import numpy as np
import sklearn
import sys
import seaborn as sns

import cv2


# import Explainer function from lime_tabular module of lime library
# from lime.lime_tabular import LimeTabularExplainer


In [ ]:

def perform_contour_analysis(dir_raw, dir_mask, dir_contour, out_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    # Initialize lists to store contour properties
    Countour_area = []
    Arc_length = []
    Covexity = []
    convexhull = []
    Name = []

    # Loop over files in dir_mask
    for file in sorted(os.listdir(dir_mask)):
        if file.endswith(".jpg"):
            print("Processing file:", file)
            # Read masked image
            mask = cv2.imread(os.path.join(dir_mask, file))
            mask[mask > 0] = 255  # Thresholding
            
            # Find contours
            contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Loop over contours
            for i, c in enumerate(contours):
                if cv2.minEnclosingCircle(c)[1] >= 50 and cv2.minEnclosingCircle(c)[1] <= 1000:
                    rect = cv2.boundingRect(c)
                    x, y, w, h = rect
                    if rect[2] <= 50 or rect[3] <= 50:
                        continue
                    if cv2.contourArea(c) >= 1000 and cv2.contourArea(c) <= 100000:
                        if cv2.arcLength(c, True) >= 240 and cv2.arcLength(c, True) <= 20000:
                            # Extract ROI
                            ROI = cv2.imread(os.path.join(dir_raw, file))[y:y+h+10, x:x+w-10]
                            
                            # Append contour properties to lists
                            Countour_area.append(cv2.minEnclosingCircle(c)[1])
                            Arc_length.append(cv2.arcLength(c, True))
                            Covexity.append(cv2.isContourConvex(c))
                            convexhull.append(cv2.convexHull(c))
                            Name.append(os.path.join(dir_contour, '{}_ROI_{}.png'.format(file.split('.')[0], i)))
                            
                            # Save ROI image
                            cv2.imwrite(os.path.join(dir_contour, '{}_ROI_{}.png'.format(file.split('.')[0], i)), ROI)
            
    # Create DataFrame with contour properties
    contour_Data = pd.DataFrame({
        "Countour_Area": Countour_area,
        "Perimeter": Arc_length,
        "Is_Convex": Covexity,
        "Convexhull": convexhull,
        "Name": Name
    })
    
    # Save DataFrame to CSV
    contour_Data.to_csv(os.path.join(out_dir, 'contour_results.csv'), index=False)

# Example usage
if __name__ == "__main__":
    dir_raw = ""  ## input directory path of raw images
    dir_mask = ""  ## output directory path of segmented image/masks
   
    dir_contour = ""   ## new dir file path
    out_dir = ""         ## new dir file path

    perform_contour_analysis(dir_raw, dir_mask, dir_contour, out_dir)


In [ ]:


### To load Ca,Age model
class ImageInceptionResNetV2(nn.Module):
    def __init__(self):
        super(ImageInceptionResNetV2, self).__init__()
        self.image_feature_extractor = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
        self.fc = nn.Sequential(
            nn.Linear(in_features=1000, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(),
            nn.Linear(in_features=128, out_features=10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        inception_outputs = self.image_feature_extractor(x)
        output = inception_outputs
        output = self.fc(output)
        return output


### To prerocess data
def get_preprocess_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])     
    transf = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])    

    return transf    

### Batch prediction
def batch_predict(model, images):
    model.eval()
    preprocess_transform = get_preprocess_transform()
    transform2 = transforms.Compose([
        transforms.ToTensor(),
    ])
    batch = torch.stack(tuple(transform2(i) for i in images), dim=0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    batch = batch.to(device)
    logits = model(batch)
    probs = logits#.argmax(dim=1)
    return probs.detach().cpu().numpy()

### For explanability module
def explanation_heatmap(exp, exp_class, output_path=None):
    dict_heatmap = dict(exp.local_exp[exp_class])
    heatmap = np.vectorize(dict_heatmap.get)(exp.segments) 
    plt.imshow(heatmap, cmap='RdBu', vmin=-heatmap.max(), vmax=heatmap.max())
    plt.colorbar()
    if output_path:
        plt.savefig(output_path)
    else:
        plt.show()

 ### Prediction_model       
def process_image_data(input_path, output_path):
    dirname = os.path.dirname(__file__)
    model_path = os.path.join(dirname, 'Model/model_epoch200.pt')
    bioactivity_model_path = os.path.join(dirname, 'Models/Bioactivity_full_trained_models/rr_TMRE_regression.sav')

    with open(os.path.join(dirname, 'Models/Bioactivity_full_trained_models/rr_TMRE_regression.sav'), 'rb') as file:
        TMRE = pickle.load(file)
        
    with open(os.path.join(dirname, 'Models/Bioactivity_full_trained_models/rr_H2DCFDA_regression.sav'), 'rb') as file:
        H2DCFDA = pickle.load(file)

    with open(os.path.join(dirname, 'Models/Bioactivity_full_trained_models/rr_FM4_46_regression.sav'), 'rb') as file:
        FM4_46 = pickle.load(file)

    with open(os.path.join(dirname, 'Models/Drugs_full_trained_model/rf_MG132_Classification.sav'), 'rb') as file:
        MG132 = pickle.load(file)
        
    with open(os.path.join(dirname, 'Models/Drugs_full_trained_model/rf_Azacytidine_Classification.sav'), 'rb') as file:
        Aza = pickle.load(file)
        
    with open(os.path.join(dirname, 'Models/Drugs_full_trained_model/rf_Etoposide_Classification.sav'), 'rb') as file:
        Etoposide = pickle.load(file)
    
    

In [ ]:
for i in os.listdir(input_path):
        
        data3 = pd.DataFrame()
        print(i)
        data = pd.read_csv(os.path.join(input_path, i))
        data2 = data.drop(["Names"], axis=1) #name col, feat col (1-128 list) - 2 cols + 1-128 col, data2 has 128 cols
        
        data2_pred = TMRE.predict(data2)
        data3["Prediction_TMRE_Intensity"] = data2_pred
        
        data2_pred = H2DCFDA.predict(data2)
        data3["Prediction_H2DCFDA_Intensity"] = data2_pred
        
        data2_pred = FM4_46.predict(data2)
        data3["Prediction_FM_Intensity"] = data2_pred
        
        data2_pred = MG132.predict(data2)
        data3["MG132"] = data2_pred
        
        data2_pred = Aza.predict(data2)
        data3["Azacytidine"] = data2_pred
        
        data2_pred = Etoposide.predict(data2)
        data3["Etoposide"] = data2_pred    

        data3["Names"] = data["Names"]
        
        item_name = "_".join(i.split("_")[2:-1])

        print("i:", i)
        print("item_name:", item_name)
        output_file = os.path.join(output_path, "predictions_" + item_name + i.split("_")[2] + '.csv')
        print(output_file) 
        data3.to_csv(output_file, index=False)

def predict_images(input_folder, output_folder, explainability=False, num_features=3, image_features=False, ko_prediction=False):
    # check_and_download_models()
    model = ImageInceptionResNetV2()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    dirname = os.path.dirname(__file__)
    model_path = os.path.join(dirname, 'model_epoch200.pt')
    model.load_state_dict(torch.load(model_path))
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # input_folder = os.path.join(output_folder, 'Test_Images')
    names_images = []
    label = []
    all_prob = []
    test_prob = []
    
    for image_file in os.listdir(input_folder):
        if image_file.endswith('.png') or image_file.endswith('.jpg'):
            image_path = os.path.join(input_folder, image_file)
            image = Image.open(image_path).convert("RGB")
            image = transform(image)
            image = image.unsqueeze(0)  # Add batch dimension

            # Move input tensor to GPU if available
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            image = image.to(device)

            with torch.no_grad():
                output = model(image)

            prediction = output.argmax(dim=1)
            label.append(prediction)
            names_images.append(image_file)

            probabilities = torch.nn.functional.softmax(output, dim=1)
            prediction = output.argmax(dim=1)
            predicted_class = prediction.item()
            predicted_probability = probabilities[0, predicted_class].item()
            all_probabilities = probabilities[0].tolist()
            test_prob.append(predicted_probability)
            all_prob.append(all_probabilities)

    print('*****************Saving Image Predictions start****************')
    Image_predictions = pd.DataFrame()
    Image_predictions["File Name"] = names_images
    Image_predictions["Label"] = pd.DataFrame(label)
    Image_predictions["Predicted_class_prob"] = test_prob
    Image_predictions["All_class_prob"] = all_prob
    Image_predictions.to_csv(os.path.join(output_folder, "Test_Image_predictions.csv"))
    print('*****************End Saving Image Predictions start****************')

    if explainability:
        print("EXPLAINABILITY")
        explainer = lime_image.LimeImageExplainer()

        def predict_fn(images):
            return batch_predict(model, images)

        for image_file in os.listdir(input_folder):
            if image_file.endswith('.png') or image_file.endswith('.jpg'):
                image = Image.open(os.path.join(input_folder, image_file)).convert("RGB")
                preprocessed_image = transform(image)
                input_image = preprocessed_image.unsqueeze(0)  # Add batch dimension

                explanation = explainer.explain_instance(input_image[0].permute(1, 2, 0).numpy(),
                                                         predict_fn,
                                                         top_labels=10,
                                                         hide_color=0,
                                                         num_samples=1000)

                transform3 = transforms.Compose([
                    transforms.Resize((299, 299)),
                    transforms.ToTensor(),
                ])
                original_image = image.copy()
                original_image  =  transform3(original_image)
                original_image  = original_image.permute(1,2,0)
                
                pdf_path1 = os.path.join(output_folder, f'{image_file}_F1_boundary1.pdf')
                fig = plt.figure(figsize=(15, 5))

                temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=50, hide_rest=False)
                img_boundry1 = mark_boundaries(np.array(original_image), mask)

                # temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True,
                #                                             num_features=50, hide_rest=False)
                # img_boundry1 = mark_boundaries(np.array(preprocessed_image.permute(1, 2, 0)), mask)
                fig = plt.figure(figsize=(15, 5))
                
                plt.imshow(img_boundry1)
                plt.title('Image with Boundaries')
                plt.savefig(pdf_path1)
                plt.close()
                
                pdf_path2 = os.path.join(output_folder, f'{image_file}_F1_boundary2.pdf')

                temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=num_features, hide_rest=False)
                plt.imshow(mark_boundaries(np.array(original_image)/ 2 + 0.5, mask))
                plt.title('Explanation Image')
                plt.savefig(pdf_path2)
                
                heatmap_path = os.path.join(output_folder, f'{image_file}_F1_heatmap.pdf')

                # heatmap_path = os.path.join(output_folder, f'{image_file}_heatmap.pdf')
                explanation_heatmap(explanation, explanation.top_labels[0], heatmap_path) #option top label upto 9
                plt.close()

    if image_features:
        print("FEATURES")
        # for i in os.listdir(main_folder):
        #     input_folder = os.path.join(main_folder, i)
            
        # Check if the item is a directory
        if os.path.isdir(input_folder):
            print("input_folder:", input_folder)

            # Initialize lists to store features and image names
            all_features = []
            all_names = []

            # Loop over all the items in the input folder
            for item in sorted(os.listdir(input_folder)):
                print("item:", item)
                item_path = os.path.join(input_folder, item)
                
                x = item_path  
                if x.endswith('.jpg') or x.endswith('.png'):
                    # Load the image
                    image_path = item_path
                    image = Image.open(image_path).convert('RGB')
                    # Apply the transformation and send the image to the device
                    input_tensor = transform(image).unsqueeze(0)
                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

                    input_tensor = input_tensor.to(device)

                    with torch.no_grad():
                        output = model.fc[0](model.image_feature_extractor(input_tensor))
                    flattened_features = torch.flatten(output)
                    
                    output = output.cpu()
                    numpy_features = output.numpy()

                    # Append features and image name to lists
                    all_features.append(list(numpy_features[0]))
                    all_names.append(item_path)

            # Create DataFrame
            column_names = ["Names"] + [f"{i+1}" for i in range(len(all_features[0]))]
            data = [[name] + features for name, features in zip(all_names, all_features)]
            formatted_feature_file = pd.DataFrame(data, columns=column_names)

            # Save to CSV
            out_path = os.path.join(output_folder, f'all_images_data_{item}.csv')
            print("Saving at:", out_path)
            formatted_feature_file.to_csv(out_path, index=False)

            print("ko prediction: ", ko_prediction)
            if ko_prediction:
                print("calling function")
                process_image_data(output_folder, output_folder)



In [ ]:

input_folder = 'Path/to/input folder'  ## output folder of perform_contour_analysis
output_folder = 'Path/to/output folder' ## user defined 


predict_images(input_folder, output_folder, explainability=True, num_features=3, image_features=True)## use False if any of the features not required


